In [1]:
from gensim import matutils
from gensim import models, corpora
from gensim.similarities import MatrixSimilarity
import numpy as np
from scipy.cluster import  hierarchy
from summa import keywords, summarizer
from pprint import pprint
import json
from TextNormalizer import TextNormalizer
from sklearn.metrics.pairwise import cosine_similarity

ModuleNotFoundError: No module named 'TextNormalizer'

Необходимо установить TextRank:

pip install summa

Также нужно положить эту тетрадку рядом c TextNormalizer.py и папкой Resources из фиджа.

Пути к сейву lsi, сейву корпуса и файлу phy_books_clear.json:

In [113]:
lsi_save_path = './out/lsi/phyge.lsi'
corpus_save_path = './out/lsi/phyge.mm'
phy_boos_path = './phy-books/phy_books.json'

Загрузили модель, корпус и phy_books:

In [52]:
lsi = models.LsiModel.load(lsi_save_path)
corpus = corpora.MmCorpus(corpus_save_path)

In [114]:
with open(phy_boos_path) as fh:
    pb = json.load(fh)

Получили распределения по тема для всего корпуса:

In [4]:
doc_vecs = lsi[corpus]

Число книг и число тем:

In [5]:
num_books = len(doc_vecs)
num_topics = len(doc_vecs[0])

Преобразовали распределения по темам в плотный нампаевский массив:

In [17]:
vv = matutils.corpus2dense(doc_vecs, num_topics).T

In [18]:
vv.shape

(1943, 300)

Провели кластеризацию:

In [137]:
threshold = 0.6
Linkage = hierarchy.linkage(vv,"average", metric="cosine")
Clusters = hierarchy.fcluster(Linkage, threshold, criterion="distance")

threshold - параметр для отсечения кластеров, если он равен, напимер, 0.6 - это значит, что в кластер будут попадать книги у которых схожесть >= 0.4.

В Clusters - лежат индексы кластера для каждой книги.


#### Важно:
Сразу предупреждаю, есть кластеры, в которые попала только одна книга, это нормально. С этим можно бороться наверное уменьшением числа тем, но тогда семантическая похожесть в кластерах может стать хуже. Также можно увеличить threshold, минусы будут такие же.

Индексы кластеров:

In [209]:
Clust_ind = set(Clusters)

Число кластеров:

In [139]:
len(Clust_ind)

368

Функция для извлкчения всего текста из описания книги:

In [186]:
def book_text(obj):
    keys = ['description', 'stikers','about_book','excerption','title']
    return '. '.join([obj[key] for key in keys]).replace('\n',' ').replace('\t',' ')

Функция для анализа кластера:

Принимает на вход:
* clust_ind - номер кластера;
* clusters - номера кластеров для каждой книги;
* books - прочитанный файл phy_books_clear.json;
* vecs - матрица, которая хранит распределения книг по темам.

Возвращает results - словарь с полями:
* books - книги в кластере (берется вся информация о кадлой книге из phy_books_clear.json);
* sims - матрица похожести книг;
* key_sent - ключевые предложения;
* key_word - ключевые слова из нормализованного текста;


#### Ключевые слова так себе получаются, лучше ориентироваться на ключевые предложения.

In [196]:
def analyze_cluster(clust_ind,clusters,books,vecs):
        result = dict()
        book_nums = np.argwhere(clusters==clust_ind).ravel()
        result['books'] = [pb[i] for i in book_nums]
        result['sims'] = cosine_similarity(np.array(vecs[book_nums]))
        books_text = '. '.join([book_text(book) for book in result['books']])
        result['key_sent'] = sorted(summarizer.summarize(books_text,language='russian',scores=True),
                                    key=lambda x: x[1],reverse=True)
        
        books_text_norm = ' '.join(TextNormalizer.normalize(books_text))
        result['key_words'] = sorted(keywords.keywords(books_text_norm,language='russian',scores=True),
                                     key=lambda x: x[1],reverse=True)
        
        return result

Пример использования:
Запустим для 50го кластера:

In [204]:
res1 = analyze_cluster(50,Clusters,pb,vv)

В него попало 5 книг:

In [210]:
[book['title'] for book in res1['books']]

['Ешь правильно, беги быстро',
 'Бегущий без сна',
 'Руководство ультрамарафонца',
 'Ваш первый марафон',
 'Рожденный бежать']

Их матрица схожести:

In [205]:
res1['sims']

array([[1.0000002 , 0.43098277, 0.4621437 , 0.45769852, 0.42785007],
       [0.43098277, 0.9999999 , 0.4777186 , 0.45078412, 0.52938867],
       [0.4621437 , 0.4777186 , 1.0000002 , 0.39095592, 0.33144084],
       [0.45769852, 0.45078412, 0.39095592, 1.        , 0.3130863 ],
       [0.42785007, 0.52938867, 0.33144084, 0.3130863 , 1.        ]],
      dtype=float32)

Ключевые предложения:

In [206]:
res1['key_sent']

[('Ультрамарафонец Дин Карназес, пробежавший 50 марафонов за 50 дней в 50 штатах (350 миль без сна), участвовавший в забеге на Южном Полюсе и победивший в легендарном испытании «Бэдуотер» (217 километров по Долине Смерти при температуре чуть ниже 50°), в своей книге отвечает на вопросы, которые ему постоянно задают: В новом издании он вдобавок ответил на несколько более практичных вопросов, которые интересовали его читателей: Эта книга стала международным бестселлером и вдохновила тысячи людей — от тех, кто совсем сторонился спорта, и до продвинутых бегунов-любителей.',
  0.19083436264426018),
 ('Из книги вы узнаете обо всем, что необходимо для подготовки к гонке: В книге есть три тренировочных плана — для ультрамарафонов на дистанциях 50 километров, 50 и 100 миль, — список из «10 вещей, которые стоит и не стоит делать в день гонки», и огромное количество советов, проверенных на практике.',
  0.1655002241601937),
 ('«Рожденный бежать» — это эпическое приключение, которое началось с про

Ключевые слова:

Они в каком-то смысле правда ключевые, но готовыми тегами их точно назвать нельзя(((

In [207]:
res1['key_words']

[('бегов', 0.226317414042905),
 ('книга', 0.2114410911710533),
 ('бежать', 0.1788156870346349),
 ('хотеть', 0.1782074435512961),
 ('хотеться', 0.1782074435512961),
 ('бег совет', 0.1586805273971284),
 ('больший', 0.14715705526749484),
 ('гонка', 0.14336168424356088),
 ('гонкой', 0.14336168424356088),
 ('человек', 0.1374175657319926),
 ('заниматься бегом', 0.13478814794983499),
 ('километр', 0.1255000512041368),
 ('ответ', 0.11449033402078403),
 ('ультрамарафонец', 0.11182646220415145),
 ('дистанция большой марафонский', 0.10982454616201191),
 ('марафон пробегать', 0.1076787056872896),
 ('бегун', 0.10285170229618423),
 ('бегать быстрый', 0.0931736753262143),
 ('подготовка', 0.09088501989883833),
 ('правило жизнь сверхмарафонец', 0.08937339292175721),
 ('подготовить ультрамарафон', 0.08355614821240528),
 ('новый', 0.08225363254951992),
 ('тренировка', 0.08189378090251044),
 ('пробежать', 0.08135451691181202),
 ('вопрос', 0.080020520793286),
 ('автор', 0.07698156463394831),
 ('день', 0.07

202й кластер:

In [216]:
res2 = analyze_cluster(202,Clusters,pb,vv)

In [217]:
[book['title'] for book in res2['books']]

['Психология убеждения', 'Лиминальное мышление']

In [218]:
res2['sims']

array([[0.99999964, 0.572231  ],
       [0.572231  , 0.9999998 ]], dtype=float32)

In [219]:
res2['key_sent']

[('Мы считаем свои убеждения совершенным представлением о мире, но на самом деле они — далеко не совершенные модели, которые помогают нам ориентироваться в сложной, многоплановой и непостижимой реальности.',
  0.24268947258230816),
 ('Убеждения — это психологическая материя, которую мы используем при создании совместного мира, где мы вместе с другими живем и работаем в соответствии со своими интересами.',
  0.2341737362759839),
 ('Авторы этой книги утверждают, что любой человек, изучая стратегии убеждения с научной точки зрения, станет в разы эффективнее общаться, а также научится честно, этично и правильно выстраивать отношение с другими людьми.',
  0.21187596603190026),
 ('Убеждения — это орудия мышления и руководства к действию, но кроме этого они могут быть искусственной преградой, за которой мы не видим имеющихся возможностей.',
  0.2010688589714655),
 ('Эта книга расскажет о психологических приемах, использование которых поможет вам на работе и в общении с близкими людьми.',
  0.

In [220]:
res2['key_words']

[('приём убеждение', 0.27772068887596624),
 ('человек', 0.2408584042499258),
 ('изменить', 0.22845778072193998),
 ('измениться', 0.22845778072193998),
 ('мир', 0.15063370613925037),
 ('психолог', 0.12058777740597593),
 ('бестселлер психология влияние книга', 0.12042819264763222),
 ('психологический', 0.10940058664798864),
 ('жизнь', 0.09098255875194168),
 ('часто', 0.09035497776175135),
 ('автор', 0.08909538088941955),
 ('хороший', 0.08542194889076357),
 ('научный', 0.08087041499578723),
 ('достигнуть', 0.07987548106845055),
 ('поразительный результат', 0.07714523738796374),
 ('большой часть прохожий', 0.07488439964425601),
 ('основа', 0.07427113323557535),
 ('научиться', 0.07308504610121465),
 ('убеждать', 0.07214313468053235),
 ('важный', 0.0708352454547702),
 ('кража', 0.07062424646179022),
 ('отношение', 0.069361741960093),
 ('использовать', 0.06842245229099507),
 ('способ убедительный', 0.06659300412787539),
 ('зрение эффективный', 0.0660464775669167),
 ('оператор', 0.065432917977

Можно провести аналитику сразу для всех, это может занять некторое время:

In [ ]:
all_results = [analyze_cluster(i,Clusters,pb,vv) for i in Clust_ind]